In [1]:
!git clone https://github.com/whnhch/cta.git

fatal: destination path 'cta' already exists and is not an empty directory.


In [2]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: u1472329 (whanhee). Use `wandb login --relogin` to force relogin


True

In [3]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
import numpy as np
from cta.utils.data import MyDataset
import sys
import random
from cta.models import TabCSE
import torch.nn as nn
import torch
import json

lr=1e-5
epochs=10
bs=64
model_name='bert-base-uncased'
max_seq=256
temp=0.05
data_path='dataset.json'

In [5]:
wandb.init(
    project="nlp-2024-tabcse",
    config={
        "batch_size": bs,
        "learning_rate": lr,
        "dataset": "table-pivot",
        "temp": temp,
    },
)

cat: /sys/module/amdgpu/initstate: No such file or directory
ERROR:root:Driver not initialized (amdgpu not found in modules)


In [6]:
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TabCSE(model_name, temp)

with open(data_path, 'r') as f:
    data = json.load(f)
mydataset = MyDataset(data['data'],data['pivot'], max_seq, tokenizer)

In [7]:
# Create a PyTorch Dataset
input_ids = torch.tensor(mydataset.x['input_ids'])
attention_masks = torch.tensor(mydataset.x['attention_mask'])

dataset = torch.utils.data.TensorDataset(input_ids,attention_masks)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=bs, shuffle=True)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
for epoch in range(epochs):
    epoch_loss=0.0

    for idx, batch in enumerate(dataloader):
        x_input_ids, x_attention_mask = batch
        x_input_ids, x_attention_mask = x_input_ids.to(device), x_attention_mask.to(device)

        z1, z2 = model(x_input_ids, x_attention_mask)

        cos_sim = model.sim(z1.unsqueeze(1), z2.unsqueeze(0))
        labels = torch.arange(cos_sim.size(0)).long().to(device)
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(cos_sim, labels)
        epoch_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (idx + 1) % 30 == 0:
            average_loss = epoch_loss / 30
            print("Epoch [{}/{}], Iteration [{}/{}], Average Loss: {:.4f}".format(epoch + 1, epochs, idx+ 1, len(dataloader), average_loss))
            wandb.log({"tabcse_loss": average_loss})
            epoch_loss = 0.0

Epoch [1/10], Iteration [30/103], Average Loss: 2.5927
Epoch [1/10], Iteration [60/103], Average Loss: 1.3456
Epoch [1/10], Iteration [90/103], Average Loss: 1.1244
Epoch [2/10], Iteration [30/103], Average Loss: 0.9146
Epoch [2/10], Iteration [60/103], Average Loss: 0.8713
Epoch [2/10], Iteration [90/103], Average Loss: 0.8504
Epoch [3/10], Iteration [30/103], Average Loss: 0.7653
Epoch [3/10], Iteration [60/103], Average Loss: 0.7500
Epoch [3/10], Iteration [90/103], Average Loss: 0.7108
Epoch [4/10], Iteration [30/103], Average Loss: 0.6551
Epoch [4/10], Iteration [60/103], Average Loss: 0.7067
Epoch [4/10], Iteration [90/103], Average Loss: 0.6618
Epoch [5/10], Iteration [30/103], Average Loss: 0.6345
Epoch [5/10], Iteration [60/103], Average Loss: 0.6235
Epoch [5/10], Iteration [90/103], Average Loss: 0.6100
Epoch [6/10], Iteration [30/103], Average Loss: 0.5985
Epoch [6/10], Iteration [60/103], Average Loss: 0.5716
Epoch [6/10], Iteration [90/103], Average Loss: 0.6182
Epoch [7/1

In [10]:
# Define a file path for saving the checkpoint
checkpoint_path = 'tabcse_checkpoint.pth'

# Create a dictionary to save the checkpoint
checkpoint = {
    'model_state_dict': model.state_dict(),
}

# Save the checkpoint to the file
torch.save(checkpoint, checkpoint_path)